# Quantum Bayesian Inference with Qiskit

##### Quantum Bayesian inference allows us to use quantum computing to perform inference on Bayesian networks, potentially leveraging quantum parallelism for more efficient computation. This tutorial will guide you through the process of using the QBayesian class to perform such inference tasks.

# Step 1: Creating a Quantum Circuit for Bayesian Network

A Bayesian network can be represented as a quantum circuit where each node is a qubit, and the edges are quantum gates that represent the conditional dependencies.